In [ ]:
import time

# 1 

#Делает из файла docx файл csv при этом берет только таблицу.

docx_path = "yourfile.docx"

import docx
import csv
import re

def extract_table_from_docx(docx_path, csv_path):
    # Открываем документ
    doc = docx.Document(docx_path)
    
    # Находим первую таблицу в документе
    if len(doc.tables) == 0:
        print("В документе нет таблиц")
        return
    
    table = doc.tables[0]
    
    # Открываем CSV файл для записи
    with open(csv_path, 'w', newline='', encoding='utf-8') as csv_file:
        csv_writer = csv.writer(csv_file)
        
        # Обходим все строки таблицы
        for row in table.rows:
            # Извлекаем текст из каждой ячейки в строке
            row_data = [cell.text.strip() for cell in row.cells]
            # Записываем строку в CSV
            csv_writer.writerow(row_data)
    
    print(f"Таблица успешно экспортирована в {csv_path}")

# Использование функции
extract_table_from_docx(docx_path, "courses_data.csv")

# 2
#РАЗБИЕНИЕ НА ГРУППЫ И ЗАНЯТИЯ, ДОБАВЛЕНИЕ ВРЕМЕНИ

import csv
import datetime
from datetime import timedelta

def parse_date(date_str):
    """Преобразует строку даты в формате ДД.ММ.ГГГГ в объект datetime."""
    return datetime.datetime.strptime(date_str, "%d.%m.%Y")

def format_date(date_obj):
    """Форматирует объект datetime в строку в формате ДД.ММ.ГГГГ."""
    return date_obj.strftime("%d.%m.%Y")

def get_weekday_name(date_obj):
    """Возвращает название дня недели на русском языке."""
    weekdays = {
        0: "Понедельник",
        1: "Вторник",
        2: "Среда",
        3: "Четверг",
        4: "Пятница",
        5: "Суббота",
        6: "Воскресенье"
    }
    return weekdays[date_obj.weekday()]

def is_day_in_schedule(date_obj, schedule):
    """Проверяет, входит ли день недели в расписание."""
    day_name = get_weekday_name(date_obj)
    
    # Проверяем на "Ежедневно" или его расшифровку
    if schedule == "Ежедневно" or "ежедневно" in schedule.lower():
        return True
    
    # Проверяем день в списке дней
    if "," in schedule:
        days = [day.strip() for day in schedule.split(",")]
        return day_name in days
    else:
        return day_name == schedule

def generate_detailed_schedule(courses):
    """Генерирует детализированное расписание на основе данных о курсах."""
    detailed_schedule = []
    
    # Преобразуем данные с учетом групп
    processed_courses = process_course_groups(courses)
    
    for course in processed_courses:
        # Извлекаем данные о курсе
        course_name = course["Название программы"]
        course_code = course["Код мероприятия"]
        academic_hours = course["Объем академ. часов"]
        
        # Форматируем полное название курса (убираем код из названия)
        full_course_name = course_name
        
        # Обрабатываем даты и время
        start_date = parse_date(course["Дата начала"])
        end_date = parse_date(course["Дата окончания"])
        start_time = course["Время начала"]
        end_time = course["Время окончания"]
        schedule = course["График"]
        
        # Определяем время суток для занятий
        time_of_day = course["Время-занятий"] if course["Время-занятий"] else "День"
        
        # Нормализуем расписание, оставляем только дни недели или "Ежедневно"
        normalized_schedule = normalize_schedule(schedule)
        
        # Добавляем общую информацию о курсе
        detailed_schedule.append({
            "Название мероприятия": full_course_name,
            "Код": course_code,
            "Дата начала обучения": course["Дата начала"],
            "Дата окончания обучения": course["Дата окончания"],
            "Время начала": start_time,
            "Время окончания": end_time,
            "Академические часы": academic_hours,
            "Формат обучения": course.get("Формат обучения", ""),  # Добавляем формат обучения
            "Примечания": normalized_schedule
        })
        
        # Генерируем детализированные строки для каждого занятия
        current_date = start_date
        lesson_number = 1
        
        while current_date <= end_date:
            if is_day_in_schedule(current_date, schedule):
                # Получаем день недели для занятия
                day_name = get_weekday_name(current_date)
                
                detailed_schedule.append({
                    "Название мероприятия": f"{full_course_name} - Занятие {lesson_number}",
                    "Код": course_code,  # Дублируем код для всех занятий
                    "Дата начала обучения": format_date(current_date),
                    "Дата окончания обучения": format_date(current_date),
                    "Время начала": start_time,
                    "Время окончания": end_time,
                    "Академические часы": "",  # Академические часы только для основной программы
                    "Формат обучения": "",  # Оставляем пустым для занятий
                    "Примечания": day_name  # Указываем день недели
                })
                lesson_number += 1
            
            current_date += timedelta(days=1)
    
    return detailed_schedule

def process_course_groups(courses):
    """Обрабатывает данные курсов с учетом групп."""
    processed_courses = []
    current_program = None
    group_counters = {}  # Счетчики групп для каждой программы
    
    for course in courses:
        program_name = course["Название программы"].strip() if course["Название программы"] else ""
        
        if program_name:  # Если имя программы указано
            # Это новая программа
            current_program = program_name
            # Инициализируем счетчик групп для этой программы
            if current_program not in group_counters:
                group_counters[current_program] = 0
            group_counters[current_program] += 1
            
            # Добавляем номер группы к названию программы
            course["Название программы"] = f"{program_name} Группа {group_counters[current_program]}"
        else:  # Если имя программы не указано
            # Это дополнительная группа текущей программы
            if current_program:
                group_counters[current_program] += 1
                course["Название программы"] = f"{current_program} Группа {group_counters[current_program]}"
        
        processed_courses.append(course)
    
    return processed_courses

def normalize_schedule(schedule):
    """Нормализует расписание, расшифровывая 'ежедневно' как полный список дней."""
    # Список допустимых дней недели
    valid_days = ["Понедельник", "Вторник", "Среда", "Четверг", "Пятница", "Суббота", "Воскресенье"]
    
    if not schedule:
        return ", ".join(valid_days)  # Пустое расписание означает все дни
    
    # Проверяем, содержит ли расписание ключевое слово "Ежедневно"
    if "ежедневно" in schedule.lower():
        return ", ".join(valid_days)  # Возвращаем полный список дней недели
    
    # Разбиваем расписание на отдельные дни
    if "," in schedule:
        days = [day.strip() for day in schedule.split(",")]
    else:
        days = [schedule.strip()]
    
    # Фильтруем только валидные дни недели
    valid_schedule_days = []
    for day in days:
        for valid_day in valid_days:
            if valid_day.lower() in day.lower():
                valid_schedule_days.append(valid_day)
                break
    
    # Если нет валидных дней, возвращаем полный список дней недели
    if not valid_schedule_days:
        return ", ".join(valid_days)
    
    # Формируем строку с днями недели
    return ", ".join(valid_schedule_days)

def read_courses_csv(file_path):
    """Читает CSV-файл с данными о курсах с учетом новой структуры столбцов."""
    courses = []
    try:
        with open(file_path, 'r', encoding='utf-8') as csvfile:
            # Определим разделитель
            sample = csvfile.read(1024)
            delimiter = ";" if ";" in sample else ","
            
            csvfile.seek(0)  # Вернуться в начало файла
            reader = csv.reader(csvfile, delimiter=delimiter)
            headers = next(reader)  # Получаем заголовки
            
            print(f"Обнаружены следующие заголовки: {headers}")
            
            # Создаем словарь соответствия ожидаемых имен столбцов и фактических заголовков
            header_mapping = {}
            
            # Обновленный список ожидаемых заголовков
            expected_headers = [
                "Название программы", 
                "Код мероприятия", 
                "Объем академ. часов", 
                "Предполагаемый объем услуг, шт.",  # Новый столбец
                "Цена за одного Слушателя, руб.",   # Новый столбец
                "Время-занятий",
                "Дата начала", 
                "Дата окончания", 
                "Время начала", 
                "Время окончания", 
                "График",           # Изменено положение
                "Формат обучения", 
                "Примечание", 
                "Преподаватель",
                "Категория обучения"
            ]
            
            # Прямое сопоставление по точному совпадению
            for expected in expected_headers:
                if expected in headers:
                    header_mapping[expected] = expected
            
            # Нечеткое сопоставление для оставшихся заголовков
            for expected in expected_headers:
                if expected not in header_mapping:
                    expected_lower = expected.lower()
                    for header in headers:
                        header_lower = header.lower()
                        
                        # Сопоставление по ключевым словам
                        if expected == "График" and "график" in header_lower:
                            header_mapping[expected] = header
                            break
                        elif expected == "Формат обучения" and "формат" in header_lower:
                            header_mapping[expected] = header
                            break
            
            # Для остальных заголовков, если не найдены по строгим критериям:
            if "Дата начала" not in header_mapping:
                for header in headers:
                    header_lower = header.lower()
                    if "дата" in header_lower and "начал" in header_lower:
                        header_mapping["Дата начала"] = header
                        break
            
            if "Дата окончания" not in header_mapping:
                for header in headers:
                    header_lower = header.lower()
                    if "дата" in header_lower and ("окон" in header_lower or "конец" in header_lower):
                        header_mapping["Дата окончания"] = header
                        break
            
            if "Время начала" not in header_mapping:
                for header in headers:
                    header_lower = header.lower()
                    if "время" in header_lower and "начал" in header_lower:
                        header_mapping["Время начала"] = header
                        break
            
            if "Время окончания" not in header_mapping:
                for header in headers:
                    header_lower = header.lower()
                    if "время" in header_lower and ("окон" in header_lower or "конец" in header_lower):
                        header_mapping["Время окончания"] = header
                        break
            
            if "Формат обучения" not in header_mapping:
                for header in headers:
                    header_lower = header.lower()
                    if "формат" in header_lower:
                        header_mapping["Формат обучения"] = header
                        break
            
            if "Примечание" not in header_mapping:
                for header in headers:
                    header_lower = header.lower()
                    if "примеч" in header_lower:
                        header_mapping["Примечание"] = header
                        break
            
            if "График" not in header_mapping:
                for header in headers:
                    header_lower = header.lower()
                    if "график" in header_lower or "период" in header_lower:
                        header_mapping["График"] = header
                        break
            
            if "Время-занятий" not in header_mapping:
                for header in headers:
                    header_lower = header.lower()
                    if "время" in header_lower and "занят" in header_lower:
                        header_mapping["Время-занятий"] = header
                        break
            
            if "Преподаватель" not in header_mapping:
                for header in headers:
                    header_lower = header.lower()
                    if "преподав" in header_lower:
                        header_mapping["Преподаватель"] = header
                        break
            
            if "Категория обучения" not in header_mapping:
                for header in headers:
                    header_lower = header.lower()
                    if "категор" in header_lower:
                        header_mapping["Категория обучения"] = header
                        break
            
            # Если всё равно не нашли ключевые заголовки, используем порядковый номер (как резервный вариант)
            if "Название программы" not in header_mapping and len(headers) > 0:
                header_mapping["Название программы"] = headers[0]
            if "Код мероприятия" not in header_mapping and len(headers) > 1:
                header_mapping["Код мероприятия"] = headers[1]
            if "Объем академ. часов" not in header_mapping and len(headers) > 2:
                header_mapping["Объем академ. часов"] = headers[2]
            if "Дата начала" not in header_mapping and len(headers) > 3:
                header_mapping["Дата начала"] = headers[3]
            if "Дата окончания" not in header_mapping and len(headers) > 4:
                header_mapping["Дата окончания"] = headers[4]
            if "Время начала" not in header_mapping and len(headers) > 5:
                header_mapping["Время начала"] = headers[5]
            if "Время окончания" not in header_mapping and len(headers) > 6:
                header_mapping["Время окончания"] = headers[6]
            if "Формат обучения" not in header_mapping and len(headers) > 7:
                header_mapping["Формат обучения"] = headers[7]
            if "Примечание" not in header_mapping and len(headers) > 8:
                header_mapping["Примечание"] = headers[8]
            if "График" not in header_mapping and len(headers) > 9:
                header_mapping["График"] = headers[9]
            if "Время-занятий" not in header_mapping and len(headers) > 10:
                header_mapping["Время-занятий"] = headers[10]
            if "Преподаватель" not in header_mapping and len(headers) > 11:
                header_mapping["Преподаватель"] = headers[11]
            if "Категория обучения" not in header_mapping and len(headers) > 12:
                header_mapping["Категория обучения"] = headers[12]
            
            # Выводим для отладки сопоставление заголовков
            print("Сопоставление заголовков:")
            for expected, actual in header_mapping.items():
                print(f"'{expected}' -> '{actual}'")
            
            # Отдельно выводим информацию о важных полях для отладки
            print("Важные сопоставления для отладки:")
            print(f"'Объем академ. часов' -> '{header_mapping.get('Объем академ. часов', 'Не найдено')}'")
            print(f"'Название программы' -> '{header_mapping.get('Название программы', 'Не найдено')}'")
            
            # Чтение строк данных
            for row in reader:
                if not any(row):  # Пропускаем пустые строки
                    continue
                    
                # Создаем словарь данных для текущей строки
                course_data = {}
                
                # Заполняем словарь значениями на основе сопоставления заголовков
                for expected_header, actual_header in header_mapping.items():
                    try:
                        index = headers.index(actual_header)
                        value = row[index] if index < len(row) else ""
                        course_data[expected_header] = value
                    except ValueError:
                        # Если заголовок не найден, используем пустое значение
                        course_data[expected_header] = ""
                
                # Заполняем обязательные поля значениями по умолчанию, если они отсутствуют
                if not course_data.get("График"):
                    course_data["График"] = "Ежедневно"
                if not course_data.get("Время-занятий"):
                    course_data["Время-занятий"] = "День"
                
                courses.append(course_data)
        
        print(f"Прочитано {len(courses)} курсов из файла.")
        return courses
        
    except Exception as e:
        print(f"Ошибка при чтении CSV-файла: {e}")
        return []

def write_detailed_schedule_csv(detailed_schedule, output_file_path):
    """Записывает детализированное расписание в CSV-файл."""
    fieldnames = [
        "Название мероприятия", 
        "Код",
        "Дата начала обучения", 
        "Дата окончания обучения", 
        "Время начала", 
        "Время окончания",
        "Академические часы", 
        "Формат обучения",  # Добавляем поле в список столбцов
        "Примечания"
    ]
    
    try:
        with open(output_file_path, 'w', encoding='utf-8', newline='') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames, delimiter=';')
            writer.writeheader()
            for row in detailed_schedule:
                writer.writerow(row)
        print(f"Детализированное расписание успешно записано в файл {output_file_path}")
    except Exception as e:
        print(f"Ошибка при записи в файл: {e}")

def create_test_data():
    """Создает тестовые данные на основе примеров из шаблона."""
    return [
        {
            "Название программы": "1. Программа",
            "Код мероприятия": "ПР-1",
            "Объем академ. часов": "24",
            "Дата начала": "21.01.2025",
            "Дата окончания": "28.02.2025",
            "Время начала": "12:00",
            "Время окончания": "15:00",
            "Формат обучения": "Дистанционная",
            "Примечание": "Онлайн",
            "График": "Ежедневно",
            "Время-занятий": "День-Вечер",
            "Преподаватель": "Иванов Иван Иванович",
            "Категория обучения": "Soft skills"
        },
        {
            "Название программы": "", # Добавили пустое значение для тестирования групп
            "Код мероприятия": "",
            "Объем академ. часов": "",
            "Дата начала": "29.01.2025",
            "Дата окончания": "04.02.2025",
            "Время начала": "12:00",
            "Время окончания": "15:00",
            "Формат обучения": "Дистанционная",
            "Примечание": "Онлайн",
            "График": "Ежедневно",
            "Время-занятий": "День-Вечер",
            "Преподаватель": "Иванов Иван Иванович",
            "Категория обучения": "Soft skills"
        },
        {
            "Название программы": "2. Обучение",
            "Код мероприятия": "",
            "Объем академ. часов": "48",
            "Дата начала": "02.03.2025",
            "Дата окончания": "04.04.2025",
            "Время начала": "09:00",
            "Время окончания": "10:30",
            "Формат обучения": "Дистанционная",
            "Примечание": "ДОТ",
            "График": "Понедельник, Вторник, Пятница",
            "Время-занятий": "",
            "Преподаватель": "Антонов Антон Антонович",
            "Категория обучения": "Soft skills"
        },
        {
            "Название программы": "3. NX CAM - программирование",
            "Код мероприятия": "NXCAM",
            "Объем академ. часов": "96",
            "Дата начала": "20.06.2025",
            "Дата окончания": "22.09.2025",
            "Время начала": "10:00",
            "Время окончания": "17:00",
            "Формат обучения": "Дистанционная",
            "Примечание": "Онлайн",
            "График": "Суббота, Воскресенье",
            "Время-занятий": "День",
            "Преподаватель": "Янов Ян Янович",
            "Категория обучения": "Hard skills"
        }
    ]

def main():
    input_file = "courses_data.csv"
    output_file = "detailed_schedule.csv"
    
    try:
        print("Чтение данных из файла...")
        courses = read_courses_csv(input_file)
        
        if not courses:
            print("Не удалось прочитать данные из файла. Использую тестовые данные из шаблона.")
            courses = create_test_data()
    except Exception as e:
        print(f"Ошибка при чтении файла: {e}")
        print("Использую тестовые данные из шаблона.")
        courses = create_test_data()
    
    print(f"Обрабатываю {len(courses)} курсов.")
    print("Генерация детализированного расписания...")
    
    detailed_schedule = generate_detailed_schedule(courses)
    
    print(f"Сгенерировано {len(detailed_schedule)} записей.")
    print("Запись результатов в файл...")
    
    write_detailed_schedule_csv(detailed_schedule, output_file)
    
    print(f"Готово! Результат сохранен в файле {output_file}")

if __name__ == "__main__":
    main()
    
    
# 3 


# создаем колонки с полной датой и поодсчет академических часов.

import pandas as pd
import re
from datetime import datetime

# Путь к файлу
path_to_file = "detailed_schedule.csv"

# Чтение CSV файла
df = pd.read_csv(path_to_file, encoding='utf-8', sep=';')

# Функция для преобразования даты и времени в формат datetime
def combine_date_time(date_str, time_str):
    if pd.isna(date_str) or pd.isna(time_str):
        return None
    
    try:
        # Формат даты: DD.MM.YYYY
        date_parts = date_str.split('.')
        if len(date_parts) != 3:
            return None
        
        day, month, year = map(int, date_parts)
        
        # Формат времени: HH:MM
        time_parts = time_str.split(':')
        if len(time_parts) != 2:
            return None
        
        hour, minute = map(int, time_parts)
        
        # Создание объекта datetime
        return datetime(year, month, day, hour, minute)
    except Exception as e:
        print(f"Error combining date and time: {e}, date: {date_str}, time: {time_str}")
        return None

# Функция для расчета академических часов
def calculate_academic_hours(start_datetime, end_datetime):
    if start_datetime is None or end_datetime is None:
        return 0
    
    try:
        # Вычисляем разницу в минутах
        diff_minutes = (end_datetime - start_datetime).total_seconds() / 60
        
        # Переводим в академические часы (1 ак. час = 45 минут)
        academic_hours = diff_minutes / 45
        
        # Округляем по стандартным правилам до целого числа
        rounded_hours = int(round(academic_hours))
        
        return rounded_hours
    except Exception as e:
        print(f"Ошибка при расчете академических часов: {e}")
        return 0

# Функция для проверки, является ли строка строкой занятия
def is_session_row(name):
    if not isinstance(name, str):
        return False
    return " - Занятие " in name or " Занятие " in name

# Функция для форматирования datetime в строку нужного формата
def format_datetime(dt):
    if dt is None:
        return None
    return dt.strftime('%d.%m.%Y %H:%M')

# Добавление объединенных столбцов даты и времени (как datetime)
df['Дата и время начала_dt'] = df.apply(
    lambda row: combine_date_time(row['Дата начала обучения'], row['Время начала']), 
    axis=1
)
df['Дата и время окончания_dt'] = df.apply(
    lambda row: combine_date_time(row['Дата окончания обучения'], row['Время окончания']), 
    axis=1
)

# Создание форматированных строковых столбцов
df['Дата и время начала'] = df['Дата и время начала_dt'].apply(format_datetime)
df['Дата и время окончания'] = df['Дата и время окончания_dt'].apply(format_datetime)

# Преобразуем существующие академические часы в целые числа для всех строк
df['Академические часы'] = df['Академические часы'].fillna(0).astype(int)

# Рассчитываем академические часы ТОЛЬКО для строк с занятиями
for i, row in df.iterrows():
    name = row["Название мероприятия"]
    
    # Проверяем, является ли строка строкой занятия
    if isinstance(name, str) and is_session_row(name):
        start = row['Дата и время начала_dt']
        end = row['Дата и время окончания_dt']
        
        if start is not None and end is not None:
            # Вычисляем академические часы и обновляем только для строк занятий
            new_hours = calculate_academic_hours(start, end)
            df.at[i, 'Академические часы'] = new_hours

# Удаляем промежуточные столбцы с datetime объектами
df = df.drop(['Дата и время начала_dt', 'Дата и время окончания_dt'], axis=1)

# Сохраняем обновленный файл
output_file = "file_with_date.csv"
df.to_csv(output_file, index=False, encoding='utf-8', sep=',')
print(f"Updated file saved to: {output_file}")

# 4 создает код мероприятия и код программы мероприйтий, тип мероприятия

path_to_file = "file_with_date.csv"

import pandas as pd
import os
import re

# Функция для извлечения номера группы из названия мероприятия
def extract_group_number(event_name):
    """
    Извлекает номер группы из названия мероприятия
    
    Args:
        event_name (str): Название мероприятия
        
    Returns:
        str: Номер группы или пустая строка, если группа не найдена
    """
    try:
        # Поиск шаблона "Группа X" или "группа X"
        match = re.search(r'[Гг]руппа\s+(\d+)', event_name)
        if match:
            return match.group(1)
        return ""
    except Exception as e:
        print(f"Ошибка при извлечении номера группы: {e}")
        return ""

# Функция для генерации кода мероприятия
def generate_event_code(event_name, start_date):
    """
    Генерирует код мероприятия по заданному шаблону
    
    Args:
        event_name (str): Название мероприятия
        start_date (str): Дата начала в формате DD.MM.YYYY
        
    Returns:
        str: Сгенерированный код мероприятия
    """
    try:
        # Извлекаем компоненты даты из формата DD.MM.YYYY
        date_parts = start_date.split('.')
        if len(date_parts) != 3:
            return 'invalid_date_format'
        
        day = date_parts[0]
        month = date_parts[1]
        year = date_parts[2][-2:]  # Последние 2 цифры года
        
        # Получаем только первые 5 слов из названия мероприятия
        words = event_name.split()
        shortened_name = ' '.join(words[:5]) if len(words) > 5 else event_name
        
        # Заменяем пробелы на подчеркивания для названия
        formatted_name = shortened_name.replace(' ', '_')
        
        # Формируем базовый код с датой
        date_code = f"{day}{month}{year}"
        
        # Извлекаем номер группы, если он есть
        group_number = extract_group_number(event_name)
        group_suffix = f"_{group_number}" if group_number else ""
        
        # Формируем код с учетом типа мероприятия и номера группы
        if "Занятие" in event_name:
            return f"H_{formatted_name}_{date_code}{group_suffix}"
        else:
            return f"tp_H_{formatted_name}_{date_code}{group_suffix}"
    except Exception as e:
        print(f"Ошибка при генерации кода: {e}")
        return "error_code"

def process_csv(path_to_file):
    """
    Обрабатывает CSV файл и добавляет столбцы "Код мероприятия", 
    "Тип мероприятия" и "Код программы мероприятий"
    
    Args:
        path_to_file (str): Путь к CSV файлу
        
    Returns:
        pandas.DataFrame: Обработанный DataFrame
    """
    try:
        # Сначала проверяем формат файла, чтобы определить правильный разделитель
        print(f"Определение разделителя в файле: {path_to_file}")
        with open(path_to_file, 'r', encoding='utf-8') as f:
            first_line = f.readline().strip()
            # Подсчитываем количество потенциальных разделителей
            commas = first_line.count(',')
            semicolons = first_line.count(';')
            
            # Выбираем разделитель на основе количества
            if semicolons > commas:
                delimiter = ';'
            else:
                delimiter = ','
        
        print(f"Обнаружен разделитель: '{delimiter}'")
        
        # Чтение CSV файла с правильным разделителем
        print(f"Чтение файла: {path_to_file}")
        df = pd.read_csv(path_to_file, encoding='utf-8', sep=delimiter)
        
        # Выводим информацию о файле
        print(f"Прочитано {len(df)} строк и {len(df.columns)} столбцов")
        print(f"Заголовки: {df.columns.tolist()}")
        
        # Проверяем наличие необходимых столбцов
        required_columns = ['Название мероприятия', 'Дата начала обучения']
        for col in required_columns:
            if col not in df.columns:
                print(f"ОШИБКА: В файле отсутствует столбец '{col}'")
                return None
        
        # 1. Добавляем новый столбец "Код мероприятия"
        print("Добавление столбца 'Код мероприятия'...")
        
        # Генерация кодов мероприятий на основе наличия слова "Занятие"
        df['Код мероприятия'] = df.apply(
            lambda row: generate_event_code(
                row['Название мероприятия'], 
                row['Дата начала обучения']
            ), 
            axis=1
        )
        
        # 2. Добавляем столбец "Тип мероприятия"
        print("Добавление столбца 'Тип мероприятия'...")
        df['Тип мероприятия'] = df['Название мероприятия'].apply(
    lambda name: "Дистанционное мероприятие" if "Занятие" in str(name)
              else "Программа мероприятий"
)
        # Обновляем столбец "Формат обучения" для программ мероприятий
        print("Обновление столбца 'Формат обучения'...")
        if "Формат обучения" not in df.columns:
            df["Формат обучения"] = ""
        else:
    # Преобразуем столбец к строковому типу
            df["Формат обучения"] = df["Формат обучения"].astype(str)
    
# Устанавливаем значение
        df.loc[df["Тип мероприятия"] == "Программа мероприятий", "Формат обучения"] = "Дистанционная"

        # 3. Добавляем столбец "Код программы мероприятий"
        print("Добавление столбца 'Код программы мероприятий'...")
        df['Код программы мероприятий'] = ""
        
        # Проходим по строкам и заполняем "Код программы мероприятий"
        last_group_code = None
        last_group_index = -1
        
        for index, row in df.iterrows():
            name = str(row['Название мероприятия'])
            code = str(row['Код мероприятия'])
            
            # Если это группа (без слова "Занятие", но с префиксом tp_H_)
            if "Занятие" not in name and code.startswith("tp_H_"):
                last_group_code = code
                last_group_index = index
                # Для групп оставляем пустую ссылку
                df.at[index, 'Код программы мероприятий'] = ""
            
            # Если это занятие (с словом "Занятие")
            elif "Занятие" in name and last_group_code is not None:
                # Если у нас есть сохраненный код группы, используем его
                df.at[index, 'Код программы мероприятий'] = last_group_code
            
            # Для других строк оставляем пустую ссылку
            else:
                df.at[index, 'Код программы мероприятий'] = ""
        
        # Сохраняем результат в новый файл
        output_path = os.path.join(os.path.dirname(path_to_file), 'raspisanie_complete_fixed.csv')
        df.to_csv(output_path, index=False, encoding='utf-8', sep=delimiter)  # Сохраняем с тем же разделителем
        print(f"Файл успешно сохранен: {output_path}")
        
        # Выводим пример первых 10 строк
        print("\nПример первых 10 строк:")
        cols_to_show = ['Название мероприятия', 'Код мероприятия', 'Тип мероприятия', 'Код программы мероприятий']
        pd.set_option('display.max_columns', None)
        pd.set_option('display.width', 1000)
        pd.set_option('display.max_colwidth', 30)
        print(df.head(10)[cols_to_show])
        
        return df
    
    except Exception as e:
        print(f"Ошибка при обработке файла: {e}")
        import traceback
        traceback.print_exc()
        return None

# Пример использования:
if __name__ == "__main__":
    # Обрабатываем файл
    result_df = process_csv(path_to_file)
    
    if result_df is not None:
        print("Обработка файла успешно завершена")
    else:
        print("Не удалось обработать файл")
        
# 5

# rename drop and new order


import pandas as pd

# Путь к файлу
path_to_file = "raspisanie_complete_fixed.csv"

# Чтение CSV файла
df = pd.read_csv(path_to_file, encoding='utf-8', sep=',')

# Удаление ненужных колонок
df = df.drop(['Дата начала обучения', 'Дата окончания обучения', 'Время начала', 'Время окончания'], axis=1)

# Переименование колонок
df = df.rename(columns={
    "Код": "Код типового мероприятия", 
    "Формат обучения": "Формат обучения (для программ)",
    "Дата и время начала": "Дата начала обучения", 
    "Дата и время окончания": "Дата окончания обучения"
})

# Создание новых пустых колонок
df["Категория обучения"] = ""
df["Контрагент"] = ""
df["Разъяснения"] = ""
df["Вид обучения"] = ""
df["Место проведения обучения"] = ""
df["Типы выдаваемых документов"] = ""
df["Номер контракта"] = ""
df["Дата заключения контракта"] = ""
df["СПУ"] = ""
df["Ссылки на курс"] = ""

df["№"] = ""

# Определение нового порядка колонок
new_order = [
    "№",
    "Название мероприятия", 
    "Код мероприятия", 
    "Код типового мероприятия", 
    "Тип мероприятия", 
    "Код программы мероприятий", 
    "Дата начала обучения", 
    "Дата окончания обучения", 
    "Категория обучения",
    "Контрагент", 
    "Разъяснения", 
    "Формат обучения (для программ)", 
    "Академические часы", 
    "Вид обучения", 
    "Место проведения обучения", 
    "Типы выдаваемых документов", 
    "Номер контракта", 
    "Дата заключения контракта", 
    "СПУ", 
    "Ссылки на курс"
]

# Проверка наличия всех колонок в DataFrame
# Если какой-то колонки нет, создаём её
for column in new_order:
    if column not in df.columns:
        df[column] = ""

# Переупорядочивание колонок
df = df[new_order]

# Если в файле есть колонки, которые не указаны в new_order, они будут потеряны.
# Если вы хотите сохранить все остальные колонки, можно использовать следующий код:
# existing_columns = [col for col in df.columns if col not in new_order]
# df = df[new_order + existing_columns]

# Сохраняем обновленный файл
# Сохраняем обновленный файл
output_file = "reordered_file.csv"
df.to_csv(output_file, index=False, encoding='utf-8-sig', sep=';')
print(f"Updated file saved to: {output_file}")